# Passio NutritionAI

为了最好地了解 NutritionAI 如何赋予您的代理超强的食品营养能力，让我们构建一个通过 Passio NutritionAI 查找该信息的代理。

## 定义工具

我们首先需要创建 [Passio NutritionAI 工具](/docs/integrations/tools/passio_nutrition_ai)。

### [Passio Nutrition AI](/docs/integrations/tools/passio_nutrition_ai)

在 LangChain 中，我们提供了一个内置工具，可以轻松使用 Passio NutritionAI 来查找食物营养成分。
请注意，这需要 API 密钥 - 他们提供免费套餐。

创建 API 密钥后，您需要将其导出为：

```bash
export NUTRITIONAI_SUBSCRIPTION_KEY="..."
```

或者通过其他方式（例如 `dotenv` 包）提供给您的 Python 环境。您也可以通过构造函数调用明确控制密钥。

In [10]:
from dotenv import load_dotenv
from langchain_core.utils import get_from_env

load_dotenv()

nutritionai_subscription_key = get_from_env(
    "nutritionai_subscription_key", "NUTRITIONAI_SUBSCRIPTION_KEY"
)

In [11]:
from langchain_community.tools.passio_nutrition_ai import NutritionAI
from langchain_community.utilities.passio_nutrition_ai import NutritionAIAPI

In [12]:
nutritionai_search = NutritionAI(api_wrapper=NutritionAIAPI())

In [ ]:
nutritionai_search.invoke("chicken tikka masala")

In [ ]:
nutritionai_search.invoke("Schnuck Markets sliced pepper jack cheese")

### 工具

现在我们有了工具，就可以创建一份我们将在下游使用的工具列表。

In [15]:
tools = [nutritionai_search]

## 创建代理

现在我们已经定义了工具，就可以创建代理了。我们将使用一个 OpenAI Functions 代理——有关此类代理以及其他选项的更多信息，请参阅[本指南](/docs/concepts/agents)

首先，我们选择想要指导代理的 LLM。

In [16]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

接下来，我们选择要用于指导代理的 prompt。

In [17]:
from langchain import hub

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

现在，我们可以使用 LLM、提示和工具初始化 agent。agent 负责接收输入并决定采取什么操作。至关重要的是，Agent 不会执行这些操作——这些操作由 AgentExecutor 执行（下一步）。有关如何理解这些组件的更多信息，请参阅我们的[概念指南](/docs/concepts/agents)

In [18]:
from langchain.agents import create_openai_functions_agent

agent = create_openai_functions_agent(llm, tools, prompt)

最后，我们将代理（大脑）与工具组合在 `AgentExecutor` 中（它将反复调用代理并执行工具）。有关如何理解这些组件的更多信息，请参阅我们的[概念指南](/docs/concepts/agents)

In [19]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

## 运行代理 (agent)

我们现在可以用几个查询来运行代理了！请注意，目前这些都是**无状态**查询（它不会记住之前的交互）。

In [20]:
agent_executor.invoke({"input": "hi!"})



> Entering new AgentExecutor chain...
Hello! How can I assist you today?

> Finished chain.


{'input': 'hi!', 'output': 'Hello! How can I assist you today?'}

In [ ]:
agent_executor.invoke({"input": "how many calories are in a slice pepperoni pizza?"})

如果想自动跟踪这些消息，我们可以将其包装在 `RunnableWithMessageHistory` 中。有关如何使用它的更多信息，请参阅[本指南](/docs/how_to/message_history)。

In [ ]:
agent_executor.invoke(
    {"input": "I had bacon and eggs for breakfast.  How many calories is that?"}
)

In [ ]:
agent_executor.invoke(
    {
        "input": "I had sliced pepper jack cheese for a snack.  How much protein did I have?"
    }
)

In [ ]:
agent_executor.invoke(
    {
        "input": "I had sliced colby cheese for a snack. Give me calories for this Schnuck Markets product."
    }
)

In [ ]:
agent_executor.invoke(
    {
        "input": "I had chicken tikka masala for dinner.  how much calories, protein, and fat did I have with default quantity?"
    }
)

## 结论

本次内容到此结束！在本快速入门指南中，我们学习了如何创建一个简单的代理，该代理能够将食物营养信息融入其回答中。代理是一个复杂的话题，有很多需要学习的地方！